In [1]:
!git clone https://github.com/myshell-ai/MeloTTS.git

%cd MeloTTS

fatal: destination path 'MeloTTS' already exists and is not an empty directory.
/content/MeloTTS


In [2]:
!pip install flask groq requests

In [3]:
!pip install cn2an pypinyin num2words pykakasi mecab-python3 unidic-lite fugashi g2p_en anyascii jamo gruut cached_path

In [4]:
!pip install gradio

In [5]:
# from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import re
import os
import requests
from groq import Groq
from melo.api import TTS

In [6]:
app = Flask(__name__)

In [7]:
# Initialize MeloTTS model
import torch
from melo.api import TTS
from melo.app import speaker_ids
device = 'auto'

model = TTS(language='EN', device=device)
speaker_ids = model.hps.data.spk2id

Make sure you've downloaded unidic (python -m unidic download) for this WebUI to work.


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/content/MeloTTS/melo/download_utils.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any us

In [8]:
# Directory to save audio files
AUDIO_DIR = 'static'
os.makedirs(AUDIO_DIR, exist_ok=True)

TEMPLATE_DIR = 'templates'
os.makedirs(TEMPLATE_DIR, exist_ok=True)

GROQ_API_KEY = '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'
groq_client = Groq(api_key=GROQ_API_KEY)

def clean_text_for_tts(text):
    # Replace or remove unwanted characters
    clean_text = re.sub(r'\*\*', '', text)  # Remove double asterisks
    clean_text = re.sub(r'\*', '', clean_text)  # Remove single asterisks
    clean_text = re.sub(r'•', '', clean_text)  # Remove bullet points
    clean_text = re.sub(r'(\d+\.)', '', clean_text)  # Remove numbers followed by a period (like "1.")
    clean_text = re.sub(r'\n', ' ', clean_text)  # Replace newlines with spaces

    # Additional replacements can be done here as needed
    return clean_text

def generate_podcast_content(prompt):
    try:
        response = groq_client.chat.completions.create(
            messages=[
                {"role": "user", "content": prompt}
            ],
            model="llama3-8b-8192"
        )
        content = response.choices[0].message.content
        return content
    except Exception as e:
        print(f"Error in generate_podcast_content: {e}")
        return "Error generating content."

def text_to_speech(text):
    try:
        # Clean the text
        cleaned_text = clean_text_for_tts(text)

        # Initialize MeloTTS model
        model = TTS(language='EN', device=device)

        # Define the path to save the audio file in the static directory
        static_audio_path = os.path.join('static', 'podcast.wav')

        # Convert text to speech
        model.tts_to_file(cleaned_text, static_audio_path, speed=1.0)

        # Return the audio file path
        return static_audio_path
    except Exception as e:
        print(f"Error in text_to_speech: {e}")
        return None

In [ ]:
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        prompt_input = request.form['prompt_input']
        content = generate_podcast_content(prompt_input)
        audio_file_path = text_to_speech(content)
        return render_template('result.html', content=content, audio_file_path=audio_file_path)
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
